In [7]:
using NearestNeighbors: KDTree, knn
using LinearAlgebra: norm


In [17]:
R_Agg = 3

X = [-3.59 -0.58 -2.45 2.567 1
-1.59 -0.58 -2.45 2.931 1
-2.59 1.15 -2.45 2.752 1
-4.59 -1.15 -0.82 2.06 1
-2.59 -1.15 -0.82 1.498 1
-0.59 -1.15 -0.82 2.871 1
-5.59 0.58 -0.82 2.694 1
-3.59 0.58 -0.82 1.122 1
-1.59 0.58 -0.82 1.805 1
-4.59 2.31 -0.82 2.874 1
-2.59 2.31 -0.82 2.502 1
-4.59 -2.31 0.82 2.874 1
-2.59 -2.31 0.82 2.502 1
-5.59 -0.58 0.82 2.694 1
-3.59 -0.58 0.82 1.122 1
-1.59 -0.58 0.82 1.805 1
-4.59 1.15 0.82 2.06 1
-2.59 1.15 0.82 1.498 1
-0.59 1.15 0.82 2.871 1
-2.59 -1.15 2.45 2.752 1
-3.59 0.58 2.45 2.567 1
-1.59 0.58 2.45 2.931 1
2.59 -0.58 -2.45 2.567 1
4.59 -0.58 -2.45 2.931 1
3.59 1.15 -2.45 2.752 1
1.59 -1.15 -0.82 2.06 1
3.59 -1.15 -0.82 1.498 1
5.59 -1.15 -0.82 2.871 1
0.59 0.58 -0.82 2.694 1
2.59 0.58 -0.82 1.122 1
4.59 0.58 -0.82 1.805 1
1.59 2.31 -0.82 2.874 1
3.59 2.31 -0.82 2.502 1
1.59 -2.31 0.82 2.874 1
3.59 -2.31 0.82 2.502 1
0.59 -0.58 0.82 2.694 1
2.59 -0.58 0.82 1.122 1
4.59 -0.58 0.82 1.805 1
1.59 1.15 0.82 2.06 1
3.59 1.15 0.82 1.498 1
5.59 1.15 0.82 2.871 1
3.59 -1.15 2.45 2.752 1
2.59 0.58 2.45 2.567 1
4.59 0.58 2.45 2.931 1]

r_max, s = 2, 1
K = 10
t, t_knn = 0, 10

(0, 10)

In [20]:
function NearNeighbor(X, t, t_knn)
    # Using kNN for Nearest Neighbors
    if t%t_knn | t == 0
        global kdtree = KDTree(X[:,1:3]')
    end 
    return kdtree
end

function force(X, r_max, s, K, kdtree)
    # Initialise displacement array
    global dX = zeros(Float64, size(X)[1], 3)

    # Loop over all cells to compute displacements
    for i in 1:size(X)[1]
        # Scan neighbours
        global idxs, _ = knn(kdtree, X[i,1:3], 14, true)

        # Initialise variables
        global Xi = X[i,1:3]
        for j in idxs
            if i != j
                global r = Xi - X[j,1:3]
                global dist = norm(r)
                # Calculate attraction/repulsion force differential here
                if dist < r_max
                    global F = - K*(dist-r_max)*(dist-r_max)*(dist - s)
                    dX[i,:] +=  r/dist * F
                end
            end
        end
    end
    return dX
end

force (generic function with 1 method)

In [33]:
@time kd = NearNeighbor(X, t, t_knn)

  0.000376 seconds (27 allocations: 5.812 KiB)


KDTree{StaticArrays.SArray{Tuple{3},Float64,1,3},Distances.Euclidean,Float64}
  Number of points: 44
  Dimensions: 3
  Metric: Distances.Euclidean(0.0)
  Reordered: true

In [38]:
@time F = force(X, r_max, s, K, kd)

  0.000928 seconds (7.19 k allocations: 346.516 KiB)


44×3 Array{Float64,2}:
  1.57531e-5   -9.07506e-5    0.000337449
 -1.57531e-5   -9.07506e-5    0.000337449
 -1.69407e-20  -0.000172509   0.000337449
  0.000103512   0.000113507  -0.000168724
 -1.69407e-20   0.000172509  -0.000337449
 -0.000119265   8.62546e-5   -0.000168724
  3.15062e-5    0.0           0.0
  0.000103512   5.90017e-5   -0.000168724
 -0.000119265   3.17489e-5   -0.000168724
  0.0          -5.45058e-5    0.0
  ⋮                          
  3.15062e-5    0.0           0.0
  0.000103512  -5.90017e-5    0.000168724
 -0.000119265  -3.17489e-5    0.000168724
  0.000103512  -0.000113507   0.000168724
 -4.93038e-31  -0.000172509   0.000337449
 -0.000119265  -8.62546e-5    0.000168724
  0.0           0.000172509  -0.000337449
  1.57531e-5    9.07506e-5   -0.000337449
 -1.57531e-5    9.07506e-5   -0.000337449